In [1]:
import os
import requests as re
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from io import BytesIO
from tqdm.auto import tqdm
import time
import glob
from datetime import datetime
import io
import zipfile

import xml.etree.ElementTree as ET

import seaborn as sns
import matplotlib.pyplot as plt

try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize

from crawling.crawler import *
from preprocessing.preprocessing_dart import DartFinstate
from crawling.crawling_dart import CrawlingDart

from database.access import AccessDataBase
from preprocessing.finance_ratio import GetRatio

## connect database
db = AccessDataBase('root', 'jys9807!', 'yeonseo')
db_bak = AccessDataBase('root', 'jys9807!', 'yeonseo_bak')

/Users/yeonseojung/Projects/finance_data/venv/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### Kospi

---
#### Crawling: dart_finstatements

In [2]:
# stock codes
stocks_df = db.get_tbl('stocks')
stock_codes = stocks_df.stock_code.unique()



`stocks` Import Time: 0.0sec


In [3]:
# Test: kospi 200
# _stock_codes = stock_codes[:200]

# Test: kospi 500
# _stock_codes = stock_codes[:500]

# All Corp
_stock_codes = stock_codes

# get finstatements to dart 
crw = CrawlingDart()

crw.quarters

['Q202211012',
 'Q202211013',
 'Y202111011',
 'Q202111014',
 'Q202111012',
 'Q202111013',
 'Y202011011',
 'Q202011014',
 'Q202011012',
 'Q202011013',
 'Y201911011',
 'Q201911014',
 'Q201911012',
 'Q201911013',
 'Y201811011',
 'Q201811014',
 'Q201811012',
 'Q201811013']

In [4]:
fins, error, error_none = [], [], []
for stock_code in tqdm(_stock_codes):
    _fins, _error, _error_none = crw.get_finstates(stock_code)
    fins += _fins
    error += _error
    error_none += _error_none
print(len(_stock_codes), len(error), len(error_none))

# re-get error: None type
fins_re_0, error_re_0 = [], []
for codeq in tqdm(error):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_0.append(fin)
    else:
        error_re_0.append([code, q])
# print(len(fins_re_0), len(error_re_0))

# re-get error: The others
fins_re_1, error_none_re_1 = [], []
for codeq in tqdm(error_none):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_1.append(fin)
    else:
        error_none_re_1.append([code, q])
print(len(fins_re_1), len(error_none_re_1))

# re-get error 2: None type
fins_re_2, error_re_2 = [], []
for codeq in tqdm(error_re_0):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_2.append(fin)
    else:
        error_re_2.append([code, q])
print(len(fins_re_2), len(error_re_2))
        
if len(error_re_0) == len(error_re_2) and len(fins_re_2) == 0:
    print('Complete!')
else:
    print("재수집이 필요합니다.")

100%|██████████| 825/825 [57:56<00:00,  4.21s/it]    


825 168 1663


100%|██████████| 1663/1663 [06:25<00:00,  4.32it/s] 


0 1663


100%|██████████| 32/32 [00:04<00:00,  7.70it/s]

0 32
Complete!


---
#### Upload: dart_finstatements

In [5]:
# concat
_fins_ = fins + fins_re_0 + fins_re_1
df_concat_kospi = pd.concat(_fins_)

# dedup
columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'fs_div', 'stock_code', 'account_id', 'account_nm', 'thstrm_nm', 'thstrm_amount']
df_dedup_kospi = df_concat_kospi.drop_duplicates(subset=columns, keep='first', ignore_index=True)

# set type
fins_df = df_dedup_kospi.loc[:, columns]
fins_df.loc[fins_df.thstrm_amount=='', 'thstrm_amount'] = np.nan
fins_df.loc[:, 'thstrm_amount'] = fins_df.thstrm_amount.astype('float64')

In [6]:
fins_df.info()
print("\n\n", "Stock code counts(unique): ", len(fins_df.stock_code.unique()), "\n\n")
fins_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728001 entries, 0 to 1728000
Data columns (total 12 columns):
 #   Column         Dtype  
---  ------         -----  
 0   rcept_no       object 
 1   reprt_code     object 
 2   bsns_year      object 
 3   corp_code      object 
 4   sj_div         object 
 5   sj_nm          object 
 6   fs_div         object 
 7   stock_code     object 
 8   account_id     object 
 9   account_nm     object 
 10  thstrm_nm      object 
 11  thstrm_amount  float64
dtypes: float64(1), object(11)
memory usage: 158.2+ MB


 Stock code counts(unique):  760 




,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,fs_div,stock_code,account_id,account_nm,thstrm_nm,thstrm_amount
0,20220816001711,11012,2022,00126380,BS,재무상태표,CFS,005930.KS,ifrs-full_CurrentAssets,유동자산,Q202211012,2.362875e+14
1,20220816001711,11012,2022,00126380,BS,재무상태표,CFS,005930.KS,ifrs-full_CashAndCashEquivalents,현금및현금성자산,Q202211012,3.958314e+13
2,20220816001711,11012,2022,00126380,BS,재무상태표,CFS,005930.KS,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,Q202211012,8.442882e+13
3,20220816001711,11012,2022,00126380,BS,재무상태표,CFS,005930.KS,-표준계정코드 미사용-,단기상각후원가금융자산,Q202211012,1.253196e+12
4,20220816001711,11012,2022,00126380,BS,재무상태표,CFS,005930.KS,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,단기당기손익-공정가치금융자산,Q202211012,5.550500e+10


In [8]:
# upload: dart_finstatements (bak)

# append
# db_bak.engine_upload(fins_df, 'dart_finstatements', 'append')

# create table 
db_bak.create_table(upload_df=fins_df, table_name="dart_finstatements")



Table upload successful: `dart_finstatements`


---
#### Upload: dart_amounts

In [9]:
''' Create quarter amount all table '''

# Kospi: All

# init
# fins_df = db_bak.get_tbl('dart_finstatements')


dartfins = DartFinstate(fins_df)

# Update Q202211012(2022__Q2)
# dartfins.quarters = ['Q202211012']

# create amount quarter
amounts_all_df = dartfins.create_amount_quarter()
amounts_all_df.groupby('sj_div').count()

# upload: dart_amounts_all (bak)

100%|██████████| 760/760 [2:50:48<00:00, 13.48s/it]  


,stock_code,corp_code,fs_div,account_id,account_nm,Q202211012,Q202211013,Y202111011,Q202111014,Q202111012,...,Q202011012,Q202011013,Y201911011,Q201911014,Q201911012,Q201911013,Y201811011,Q201811014,Q201811012,Q201811013
sj_div,,,,,,,,,,,,,,,,,,,,,
BS,91166,91166,91166,56519,91166,49499,49131,50281,49188,48975,...,47856,47770,48694,46527,46326,45657,45476,44137,44216,44067
CF,109461,109461,109461,49915,109461,45228,41128,51095,47030,44888,...,44046,40571,48866,44402,42944,38993,47433,43575,41993,39559
CIS,53159,53159,53159,29628,53159,24382,23871,25803,24691,24518,...,24535,24386,26077,24327,24212,23958,25623,24323,24356,24540
IS,5500,5500,5500,3465,5500,2940,3000,2956,2981,2896,...,2845,2823,2863,2784,2798,2780,2815,2794,2839,2743


In [10]:
# upload: dart_amounts_all (bak)

# Create table
db_bak.create_table(upload_df=amounts_all_df, table_name='dart_amounts_all')



Table upload successful: `dart_amounts_all`


In [13]:
''' Get confirmed account amount 
    All data (replace) '''

dartfins = DartFinstate(pd.DataFrame())
amounts_all_df = db_bak.get_tbl('dart_amounts_all')
accounts_df = db.get_tbl('accounts')
amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)
amounts_df.groupby('account_nm_eng').count()['stock_code']



`dart_amounts_all` Import Time: 12.5sec


`accounts` Import Time: 0.0sec


100%|██████████| 760/760 [06:34<00:00,  1.93it/s]  


account_nm_eng
administrative_expenses    723
assets                     760
cash_equivalents           759
cost_of_sales              683
current_assets             719
current_liabilities        719
equity                     757
income                     759
interest_expense            63
interest_income             78
investories                694
issued_capital             705
liabilities                719
net_profit                 717
non_controlling_equity     619
non_controlling_profit     573
other_gains                730
other_losses               708
owner_equity               624
owners_profit              566
profit_before_tax          720
profit_continuing          213
quick_assets               532
revenue                    709
tax_expense                715
Name: stock_code, dtype: int64

In [14]:
''' Caculate 4th quarter net '''

df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)

# create date, regist date
df_amounts_quarter = db.set_date(df_amounts_quarter)

df_amounts_quarter.info()
df_amounts_quarter.tail(5)

100%|██████████| 12243/12243 [00:13<00:00, 894.62it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12243 entries, 0 to 12242
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   stock_code      12243 non-null  object        
 1   fs_div          12243 non-null  object        
 2   sj_div          12243 non-null  object        
 3   account_id      12243 non-null  object        
 4   account_nm_eng  12243 non-null  object        
 5   account_nm_kor  12243 non-null  object        
 6   Q202211012      12235 non-null  float64       
 7   Q202211013      12028 non-null  float64       
 8   Q202111011      11894 non-null  float64       
 9   Q202111014      11963 non-null  float64       
 10  Q202111012      11894 non-null  float64       
 11  Q202111013      11794 non-null  float64       
 12  Q202011011      11666 non-null  float64       
 13  Q202011014      11654 non-null  float64       
 14  Q202011012      11601 non-null  float64       
 15  Q2

,stock_code,fs_div,sj_div,account_id,account_nm_eng,account_nm_kor,Q202211012,Q202211013,Q202111011,Q202111014,...,Q201911011,Q201911014,Q201911012,Q201911013,Q201811011,Q201811014,Q201811012,Q201811013,created,updated
12238,204210.KS,OFS,BS,ifrs-full_CurrentLiabilities,current_liabilities,유동부채,5.016900e+10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-16 18:25:45.949346,2022-12-16 18:25:45.950410
12239,204210.KS,OFS,BS,ifrs-full_Liabilities,liabilities,부채,5.034630e+10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-16 18:25:45.949346,2022-12-16 18:25:45.950410
12240,204210.KS,OFS,BS,ifrs-full_IssuedCapital,issued_capital,자본금,3.913410e+10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-16 18:25:45.949346,2022-12-16 18:25:45.950410
12241,204210.KS,OFS,BS,ifrs-full_TradeAndOtherCurrentReceivables,quick_assets,당좌자산,1.687610e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-16 18:25:45.949346,2022-12-16 18:25:45.950410
12242,204210.KS,OFS,BS,ifrs-full_CashAndCashEquivalents,cash_equivalents,현금및현금성자산,1.687550e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-16 18:25:45.949346,2022-12-16 18:25:45.950410


In [17]:
def bak_trunc(table):
    bak_date = datetime.today().strftime("%y%m%d")
    bak_query = f"CREATE TABLE IF NOT EXISTS `{table}_{bak_date}` SELECT * FROM `{table}`;"
    db._execute(bak_query)
    trunc_query = f'TRUNCATE TABLE {table};'
    db._execute(trunc_query)
    
# bak_trunc("amounts")

In [18]:
''' Insert data: amounts (service) '''

table = 'amounts'
fields = tuple(df_amounts_quarter.columns)
data = df_amounts_quarter.values.tolist()
db.insert_many(table_name=table, fields=fields, data=data)

# db.engine_upload(upload_df=df_amounts_quarter_db, table_name=table, if_exists_option='append')

In [ ]:
''' Upload table: dart_amounts (backup) '''

# df_amounts_quarter = db.get_tbl("amounts")
db_bak.create_table(upload_df=df_amounts_quarter, table_name="dart_amounts")

---
#### table delete | truncate | dedup

In [ ]:
# tables = ['dart_amounts', 'dart_accounts']
# tables = ['dart_accounts']
# app_name = 'django'

In [ ]:
# Drop

conn, curs = db._connect()

for table in tables:
    query = f'DROP TABLE {table};'
    curs.execute(query)

query = f"DELETE FROM `{app_name}_migrations` WHERE app='{app_name}';"
curs.execute(query)

conn.commit()
curs.close()
conn.close()

In [ ]:
# Truncate

tables = ["amounts"]
conn, curs = db._connect()

for table in tables:
    query = f'TRUNCATE TABLE {table};'
    curs.execute(query)

conn.commit()
curs.close()
conn.close()

In [ ]:
# dedup query: `yeonseo.amounts`

dedup_query = f"""
    DELETE t1
    FROM amounts t1, amounts t2
    WHERE 
        t1.stock_code=t2.stock_code and
        t1.account_nm_eng=t2.account_nm_eng and
        t1.id < t2.id; 
"""

db._execute(dedup_query)

In [ ]:
# dedup query: `yeonseo.accounts`

dedup_query = f"""
    DELETE t1
    FROM accounts t1, accounts t2
    WHERE 
        t1.account_id=t2.account_id and
        t1.id < t2.id; 
"""

db._execute(dedup_query)

---
### Kosdaq

In [ ]:
# today 
today_date = datetime.today().strftime('%Y%m%d')

# market
market = "kosdaq"

# 주가정보, 거래대금 및 주식 수 정보
stocks = get_stock_info(market, today_date)
    
# 보통주 종목코드 
common =  get_common_stock_info(market)

# stocks에서 보통주만 추출 
common_stocks = common.loc[:, ['종목코드']].merge(stocks, on='종목코드', how='inner').sort_values(by='시가총액', ascending=False).reset_index(drop=True)

# stock codes
stock_codes = common_stocks.종목코드.unique().tolist()


In [ ]:
# get finstatements to dart 
crw = CrawlingDart()

fins, error, error_none = [], [], []
for stock_code in tqdm(stock_codes):
    _fins, _error, _error_none = crw.get_finstates(stock_code)
    fins += _fins
    error += _error
    error_none += _error_none
print(len(stock_codes), len(error), len(error_none))

# re-get error: None type
fins_re_0, error_re_0 = [], []
for codeq in tqdm(error):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_0.append(fin)
    else:
        error_re_0.append([code, q])
print(len(fins_re_0), len(error_re_0))

# re-get error: The others
fins_re_1, error_none_re_1 = [], []
for codeq in tqdm(error_none):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_1.append(fin)
    else:
        error_none_re_1.append([code, q])
print(len(fins_re_1), len(error_none_re_1))


# concat & dedup
_fins_ = fins + fins_re_0 + fins_re_1
df_concat_kosdaq = pd.concat(_fins_)
df_dedup_kosdaq = df_concat_kosdaq.drop_duplicates(keep='first')

---
### Test

In [ ]:
''' Test 1: Create Table '''

# conn, curs = db_bak._connect()

# query = '''\
# CREATE TABLE `dart_accounts_test` (\
# `id` bigint AUTO_INCREMENT NOT NULL PRIMARY KEY,\
# `account_nm_eng` varchar(255) NOT NULL,\
# `account_id` varchar(255) NOT NULL,\
# `account_nm_kor` varchar(255) NOT NULL);'''

# curs.execute(query)

# conn.commit()
# curs.close()
# conn.close()

table='dart_accounts_test'
fields=('account_nm_eng', 'account_id', 'account_nm_kor')

for i in range(len(accounts_df)):
    value = tuple(accounts_df.loc[i].values)
    db_bak.insert(table=table, fields=fields, values=value)

In [ ]:
# ''' Test 2: finstatement all '''
# df = db_bak.get_tbl('dart_finstatements')

# code = df.sample(1).stock_code.values[0]
# sj_div = 'BS'
# thstrm_nm = 'Y202111011'

Downloads = '/Users/yeonseosla/Downloads'
df.loc[(df.stock_code==code) & (df.sj_div=='BS') & (df.thstrm_nm=='Y202111011')].to_csv('/Users/yeonseosla/Downloads/bs.csv', index=False)

In [ ]:
''' Test 3: update_amounts '''

def update_amounts(accounts_df, amounts_all_df):
    ''' Get confirmed account amount 
        New data (append) '''

    dart_amounts_df = db.get_tbl('dart_amounts')
    confirmed_accounts = dart_amounts_df.account_nm_eng.unique().tolist()

    new_accounts = []
    for account in accounts_df.account_nm_eng.unique():
        if account in confirmed_accounts:
            pass
        else:
            new_accounts.append(account)
    accounts_df = accounts_df[accounts_df.account_nm_eng.isin(new_accounts)].reset_index(drop=True)

    if accounts_df.empty:
        status = 0
        accounts_new = None
        print('Dataframe empty')
    else:
        status = 1
        accounts_new = accounts_df.account_nm_eng.unique().tolist()
        amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)
        
    if status == 1:
        # Caculate 4th quarter net
        df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
        df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)
        df_amounts_quarter_db = df_amounts_quarter.drop(columns=['corp_code'])

        # Upload table: dart_amounts
        table = 'dart_amounts'
        fields = tuple(df_amounts_quarter_db.columns)
        data = df_amounts_quarter_db.values.tolist()
        db.insert_many(table_name=table, fields=fields, data=data)

    return status, accounts_new

In [ ]:
accounts_df = db_bak.get_tbl('dart_accounts')
dart_amounts_df = db.get_tbl('dart_amounts')
confirmed_accounts = dart_amounts_df.account_nm_eng.unique().tolist()

new_accounts = []
for account in accounts_df.account_nm_eng.unique():
    if account in confirmed_accounts:
        pass
    else:
        new_accounts.append(account)
accounts_df = accounts_df[accounts_df.account_nm_eng.isin(new_accounts)].reset_index(drop=True)
accounts_df

In [ ]:
# accounts_new = accounts_df.account_nm_eng.unique().tolist()
# dartfins = DartFinstate(pd.DataFrame())
# amounts_all_df = db_bak.get_tbl('dart_amounts_all')
# amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)

# Caculate 4th quarter net
# df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
# df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)
df_amounts_quarter_db = df_amounts_quarter.drop(columns='corp_code')

---
### ROE

In [ ]:
# df_amounts_quarter = db.get_tbl('dart_amounts')
# quarters = DartFinstate(pd.DataFrame()).quarters_q

# df_amounts_annualized = calculate_annualized(quarters_i=4) # annualized
df_amounts_annualized_octa = calculate_annualized(quarters_i=8) # annualized_octa

In [ ]:
# db_bak.create_table(upload_df=df_amounts_annualized, table_name='dart_annualized')
# db_bak.create_table(upload_df=df_amounts_annualized_octa, table_name='dart_annualized_octa')

In [ ]:
# ROE 계산을 위한 필수 계정과목 금액 존재하는 종목만 추출
# essential_accounts = ['revenue', 'assets', 'equity', 'tax_expense', 'profit_before_tax', 'income', 'owner_equity']
essential_accounts = ['owners_profit', 'owner_equity']

codes = []
for code in df_amounts_annualized_octa.stock_code.unique():
    accounts = df_amounts_annualized_octa.loc[df_amounts_annualized_octa.stock_code==code, 'account_nm_eng'].tolist()
    status = 1
    for account in essential_accounts:
        if account not in accounts:
            status = 0
    
    if status == 1:
        codes.append(code)
df_amounts_essential = df_amounts_annualized_octa.loc[df_amounts_annualized_octa.stock_code.isin(codes)].reset_index(drop=True)
df_amounts_essential = df_amounts_essential[df_amounts_essential.Q202211013.notnull()].reset_index(drop=True)
print('stock_code counts: ', len(df_amounts_essential.stock_code.unique()))

In [ ]:
# def get_roe(df_amounts_annual, stock_code):
#     ''' Get roe (std) '''
    
#     _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
#     accounts = _df_corp.account_nm_eng.unique().tolist()
    
#     # 지배지분귀속 당기순이익
#     owners_profit = _df_corp.loc[_df_corp.account_nm_eng=='owners_profit'].iloc[0, 7:]

#     # 지배주주지분
#     owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]
    
#     # roe
#     ROE = owners_profit / owner_equity

#     roe_corp = pd.DataFrame([owners_profit, owner_equity, ROE]).reset_index(drop=True)

#     roe_corp.loc[:, 'stock_code'] = stock_code
#     i = 0
#     for indicator in ['owners_profit', 'owner_equity', 'ROE']:
#         roe_corp.loc[i, 'ratio'] = indicator
#         i += 1
        
#     return roe_corp

In [ ]:
def get_roe(df_amounts_annual, stock_code):
    ''' Get roe (3 step) 
        ROE = NPM * Asset Turnover * Equity Multiplier

        - NPM (Net Profit Margin, the measure of operating efficiency)
            
            지배지분귀속 당기순이익 / 매출 
            
        - Asset Turnover (the measure of asset use efficiency)
            
            매출 / 자산
            
        - Equity Multiplier (the measure of financial leverage)
            
            자산 / 자기자본
    ''' 
    
    _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
    accounts = _df_corp.account_nm_eng.unique().tolist()
    
    # 지배지분귀속 당기순이익
    owners_profit = _df_corp.loc[_df_corp.account_nm_eng=='owners_profit'].iloc[0, 7:]
    
    # 매출액
    revenue = _df_corp.loc[_df_corp.account_nm_eng=='revenue'].iloc[0, 7:]
    
    # 자산
    assets = _df_corp.loc[_df_corp.account_nm_eng=='assets'].iloc[0, 7:]
    
    # 지배주주지분
    owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]
    
    # NPM
    NPM = owners_profit / revenue
    
    # AT
    AT = revenue / assets
    
    # EM 
    EM = assets / owner_equity
    
    # ROE 3 step
    ROE = NPM * AT * EM

    roe_corp = pd.DataFrame([owners_profit, revenue, assets, owner_equity, NPM, AT, EM, ROE]).reset_index(drop=True)

    roe_corp.loc[:, 'stock_code'] = stock_code
    i = 0
    for indicator in ['owners_profit', 'revenue', 'assets', 'owner_equity', 'NPM', 'AT', 'EM', 'ROE']:
        roe_corp.loc[i, 'ratio'] = indicator
        i += 1
        
    return roe_corp

In [ ]:
# def get_roe(df_amounts_annual, stock_code):
#     ''' Get roe (step 5) '''
    
#     _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
#     accounts = _df_corp.account_nm_eng.unique().tolist()
    
#     ## IS ##
#     # 매출액
#     revenue = _df_corp.loc[_df_corp.account_nm_eng=='revenue'].iloc[0, 7:]
#     # 법인세비용
#     tax_expense = _df_corp.loc[_df_corp.account_nm_eng=='tax_expense'].iloc[0, 7:]
#     # 세전계속사업이익
#     profit_before_tax = _df_corp.loc[_df_corp.account_nm_eng=='profit_before_tax'].iloc[0, 7:]
    
#     # 매출원가
#     if 'cost_of_sales' in accounts:
#         cost_of_sales = _df_corp.loc[_df_corp.account_nm_eng=='cost_of_sales'].iloc[0, 7:]
#     else:
#         cost_of_sales = 0
    
#     # 판매비와관리비
#     if 'administrative_expenses' in accounts:
#         administrative_expenses = _df_corp.loc[_df_corp.account_nm_eng=='administrative_expenses'].iloc[0, 7:]
#     else:
#         administrative_expenses = 0
    
#     # 이자비용
#     if 'interest_expense' in accounts:
#         interest_expense = _df_corp.loc[_df_corp.account_nm_eng=='interest_expense'].iloc[0, 7:]
#     else:
#         interest_expense = 0
    
#     # 영업외손익
#     # if 'other_gains' in accounts:
#     #     other_gains = _df_corp.loc[_df_corp.account_nm_eng=='other_gains'].iloc[0, 7:]
#     # else:
#     #     other_gains = 0

#     ## BS
#     assets = _df_corp.loc[_df_corp.account_nm_eng=='assets'].iloc[0, 7:]
#     equity = _df_corp.loc[_df_corp.account_nm_eng=='equity'].iloc[0, 7:]
#     owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]


#     '''- OPM (Operting Profit Magin)
#         EBIT / revenue
        
#         EBIT = revenue - cost_of_sales - administrative_expenses + other_gains - other_losses
#             = 당기순이익 + 이자비용 + 법인세비용 - 중단영업이익 + 중단영업손실'''

#     EBIT = revenue - cost_of_sales - administrative_expenses
#     if EBIT.sum() == revenue.sum():
#         EBIT = _df_corp.loc[_df_corp.account_nm_eng=='income'].iloc[0, 7:]
#     else:
#         pass
#     OPM = EBIT / revenue

#     '''- AT (Asset Turnover)
#         매출 / 자산'''
#     AT = revenue / assets

#     '''- IER (Interst Expense Rate)
#         이자비용 / 자산'''
#     IER = interest_expense / assets

#     '''- EM (Equity Multiplier)
#         자산 / 자기자본'''
#     EM = assets / equity      
        
#     '''- TRR (Tax Retention Rate)
#         1 - 법인세율'''
        
#     TRR = 1 - tax_expense / profit_before_tax

#     '''- ROE (Return On Equity)
#         (OPM * AT - IER) * EM * TRR'''
#     ROE = (OPM * AT - IER) * EM * TRR

#     roe_corp = pd.DataFrame([OPM, AT, IER, EM, TRR, ROE, owner_equity]).reset_index(drop=True)

#     roe_corp.loc[:, 'stock_code'] = stock_code
#     i = 0
#     for indicator in ['OPM', 'AT', 'IER', 'EM', 'TRR', 'ROE', 'owner_equity']:
#         roe_corp.loc[i, 'ratio'] = indicator
#         i += 1
        
#     return roe_corp

In [ ]:
stock_codes = df_amounts_essential.stock_code.unique()
df_list, error = [], []
for stock_code in tqdm(stock_codes):
    
    try:
        roe_corp = get_roe(df_amounts_essential, stock_code)
        df_list.append(roe_corp)
    except IndexError:
        # 필수 계정과목이 누락된 경우
        error.append(stock_code)
    
roe_df = pd.concat(df_list).reset_index(drop=True)

---
### RIM

- corp_val = equity + equity * (roe - r) * (w / (1 + r - w))  
- price_sell = (equity + equity * (roe - r) / r) / stock_num (w = 1)  
- price_prop = (equity + equity * (roe - r) * 0.9 / (1 + r - 0.9)) / stock_num (w = 0.9)  
- price_buy = (equity + equity * (roe - r) * 0.8 / (1 + r - 0.8)) / stock_num (w = 0.8)  

In [ ]:
stocks_df = db.get_tbl('stocks')
stock_codes = roe_df.stock_code.unique()
columns = roe_df.columns[:10]
n = len(columns)
values_df = stocks_df.copy()

rate = 0.08
roe_df_ = roe_df[(roe_df.Q202211013>rate) & (roe_df.ratio=='ROE')]
stock_codes = roe_df_.stock_code.unique()

In [ ]:
for stock_code in tqdm(stock_codes):
    roe = roe_df.loc[(roe_df.stock_code==stock_code) & (roe_df.ratio=='ROE'), columns].values[0]
    equity = roe_df.loc[(roe_df.stock_code==stock_code) & (roe_df.ratio=='owner_equity'), columns].values[0]
    shares = stocks_df.loc[stocks_df.stock_code==stock_code, 'shares'].values[0]
    
    # buy price
    weight = 0.8
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_buy = corp_val / shares
    columns_buy = columns + '_buy'
    values_df.loc[values_df.stock_code==stock_code, columns_buy] = price_buy
    
    # sell price
    weight = 1
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_sell = corp_val / shares
    columns_sell = columns + '_sell'
    values_df.loc[values_df.stock_code==stock_code, columns_sell] = price_sell
    
    # prop price
    weight = 0.9
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_prop = corp_val / shares
    columns_prop = columns + '_prop'
    values_df.loc[values_df.stock_code==stock_code, columns_prop] = price_prop

In [ ]:
roe_df_

In [ ]:
# values_df.loc[:, 'buy_ratio'] = (values_df.Q202211013_buy - values_df.close) / values_df.Q202211013_buy

min_buy_ratio = 0.5
values_df.loc[values_df.buy_ratio>min_buy_ratio].sort_values('buy_ratio', ascending=False, ignore_index=True)

In [ ]:
values_df.info()

In [ ]:
sns.kdeplot((values_df.Q202211013_buy - values_df.close) / values_df.Q202211013_buy)

In [ ]:
sns.kdeplot(values_df.loc[values_df.Q202211013_buy > 0, 'Q202211013_buy'])

---
### Finanace ratio

In [ ]:
ratios = [
    'impaired_capital', # 자본잠식률 = (자본금 - 지배주주지분) / 자본금
    'bis_ratio', # 자기자본비율 = 자본총계 / 자산총계
    'current_ratio', # 유동비율 = 유동자산 / 유동부채
    'quick_ratio', # 당좌비율 = 당좌자산 / 유동부채
    '', # 수중유동성 = (현금, 예금, 유가증권, 즉시 현금화 가능 자산, 즉시 조달가능 금액) / 월매출(최근 12개월 평균)
    '', # D/E 비율 = 유이자부채 / 자기자본
    'ROE', # ROE = 지배지분귀속 당기순이익 (8분기 연율화) / 자기자본 (전기, 당기 평균)
]

In [ ]:
getr = GetRatio() 

def get_ratios_df(ratio, expressions):
    ratios, errors = getr.get_ratios(ratio, expressions)

    quarters = DartFinstate(pd.DataFrame()).quarters_q
    columns = ['stock_code', 'ratio'] + quarters
    ratio_df = pd.DataFrame(ratios, columns=columns)
    # ratio_df.sort_values(by='Q202211013', ignore_index=True)
    
    return ratio_df

In [ ]:
ratio_expression = {
    'impaired_capital': 'issued_capital -- equity ++ non_controlling_equity / issued_capital', # 자본잠식률 = (자본금 - 지배주주지분) / 자본금
    'bis_ratio': 'equity / assets', # 자기자본비율 = 자본총계 / 자산총계
    'currnet_ratio': 'current_assets / current_liabilities', # 유동비율 = 유동자산 / 유동부채
    # 'quick_ratio': 'quick_assets / current_liabilities', # 당좌비율 = 당좌자산 / 유동부채
    'quick_ratio': 'current_assets -- investories / current_liabilities', # 당좌비율 = (유동자산 - 재고자산) / 유동부채
    # 'cash_liquidity': 'quick_assets / revenue', # 수중유동성 = (현금, 예금, 유가증권, 즉시 현금화 가능 자산, 즉시 조달가능 금액) / 월매출(최근 12개월 평균)
}

df_list = []
for ratio in ratio_expression.keys():
    expression = ratio_expression[ratio]

    # save
    ratio_expression[ratio] = expression
    expressions = expression.split(' ')
    ratio_df = get_ratios_df(ratio, expressions)
    df_list.append(ratio_df)
    
ratios_df = pd.concat(df_list, ignore_index=True)
ratios_df.groupby('ratio').count()

In [ ]:
# data = ratio_df.sort_values(by='Q202211013', ignore_index=True)['Q202211013']
# sns.kdeplot(data)

In [ ]:
""" Insert data: ratios (service) """

ratios_df = db.set_date(ratios_df)
fields = tuple(ratios_df.columns)
data = ratios_df.values.tolist()

db.insert_many(table_name='ratios', fields=fields, data=data)

In [ ]:
# Check insert
ratios_df = db.get_tbl('ratios')
ratios_df.groupby('ratio').count()

In [ ]:
## Table Backup ##

# ratios_df = db.get_tbl('dart_ratios')
# db_bak.create_table(upload_df=ratios_df, table_name='dart_ratios')

In [ ]:
# ## Delete data ##

# conn, curs = db._connect()
# table = 'dart_ratios'

# # Enter the WHERE condition
# where_condition = "ratio='quick_ratio'"

# query = f'DELETE FROM {table} WHERE {where_condition};'

# curs.execute(query)
# conn.commit()

# curs.close()
# conn.close()

---
### Stocks data

In [ ]:
def get_stocks(market: str):
    
    # today 
    today_date = datetime.today()
    _today_date = today_date.strftime('%Y%m%d')

    stocks = None
    while stocks is None:
        try:
            # 주가정보, 거래대금 및 주식 수 정보
            stocks = get_stock_info(market, _today_date)
            print(f"Date is {_today_date}")
        except ValueError:
            today_date += timedelta(days=-1)
            _today_date = today_date.strftime('%Y%m%d')
        
    # 보통주 종목코드 
    common =  get_common_stock_info(market)

    # stocks에서 보통주만 추출 
    common_stocks = common.loc[:, ['stock_code']].merge(stocks, on='stock_code', how='inner').sort_values(by='market_cap', ascending=False).reset_index(drop=True)
    # Set type: float
    common_stocks = pd.concat([common_stocks[common_stocks.columns[:3]], common_stocks[common_stocks.columns[3:]].astype("float")], axis=1)
    common_stocks = db.set_date(common_stocks)

    return common_stocks

In [ ]:
def update_stocks(market: str):

    common_stocks = get_stocks(market)
    # get exist stock codes
    query = """SELECT DISTINCT(`stock_code`) FROM `stocks`"""
    data = db._execute(query)
    stock_codes = pd.DataFrame(data).stock_code.tolist()
    
    fields = common_stocks.columns.tolist()
    
    # update data
    new_stocks, exist_stocks = [], []
    ranges = range(len(common_stocks))
    for idx in tqdm(ranges):
        stock_code, stock_name, market, close, open, high, low, volume, amounts, market_cap, shares, created, updated = common_stocks.loc[idx, :].values

        if stock_code in stock_codes:
            exist_stocks.append(stock_code)
            query = f'\
            UPDATE `stocks` \
            SET close={close}, open={open}, low={low}, volume={volume}, amounts={amounts}, market_cap={market_cap}, shares={shares}, updated="{updated}" \
            WHERE stock_code="{stock_code}";'
            conn, curs = db._connect()
            curs.execute(query)
            conn.commit()

        else: 
            # query = f'\
            # INSERT INTO `stocks`(stock_code, stock_name, market, close, open, high, low, volume, amounts, market_cap, shares, created, updated) \
            # VALUES ({stock_code}, {stock_name}, {market}, {close}, {open}, {high}, {low}, {volume}, {amounts}, {market_cap}, {shares}, {created}, {updated});'
            new_stocks.append(stock_code)
            values = tuple(stock_code, stock_name, market, close, open, high, low, volume, amounts, market_cap, shares, created, updated)
            db.insert("stocks", fields, values)
        
    curs.close()
    conn.close()
    
    print(f"\
        - updated stocks: {len(exist_stocks)}\n\
        - new stocks: {len(new_stocks)}"
        )
    
    return exist_stocks, new_stocks

In [ ]:
exist_stocks, new_stocks = update_stocks("kospi")

In [ ]:
stocks_df = db.get_tbl('stocks')
stocks_df.sort_values('amounts', ascending=False).head(60)